# Using the Banana Command Line Interface

On top of the Arcana's Python API, Banana adds a command line interface to make it easier to navigate the extensive analyses implemented within it.

All commands start with `banana` followed by a sequence of commands and switches depending on what you would like to do. The best place to start is with the `help` switch to find out about the options available.

In [ ]:
! banana help

You can also get help for a specific command by appending it to the help command

In [ ]:
! banana help menu

### Avail

A useful command is `avail` which lists the Analysis classes that are implemented in Banana. Note that you can use the banana command line with other Analysis classes not implemented in banana (i.e. that you have written yourself or received from a third-party), they just have to be on your [Python Path](https://www.devdungeon.com/content/python-import-syspath-and-pythonpath-tutorial).

In [ ]:
! banana avail

### Menu

As the inputs, derivatives and parameters for Analysis classes in Banana are typically spread over several base-classes, the `menu` command is very useful for getting an overview of the capabilities of a given Analysis class.

In [ ]:
! banana menu mri.DwiAnalysis

### Derive

Once you have decided on the Analysis class you want to use, you use the `derive` command to derive one or more derivatives from it. See the help menu for full options

In [ ]:
! banana help derive

We will use the `derive` command to derive a brain mask from the T1-weighted scans in the [BIDS formatted](https://bids.neuroimaging.io) dataset `ds000114`. Since it takes ~5 mins for each Brain extraction we will only run it on subject `01` and visit `test`. Since BIDS strictly defines the names of scans for each contrast Banana knows where to look for the T1-weighted scans and we don't need to provide any input filters (unlike with Plain and XNAT datasets).

**IMPORTANT** When running on MASSIVE/CVL it is important to only use the "project" and "scratch" directories to store data (as otherwise the home directory fills up). For working data we won't want to use again we should use the project scratch space '~/fs97_scratch' (which is syminked to 'work' in the CWD). We do this by providing the path to the working directory to the '--scratch' option.

Since we are only processing one subject we use the `--processor single` option to select a `SingleProc` processor. For multiple subjects you will either want use the default multi-processor or submit to MASSIVE's SLURM scheduler with the `slurm` processor.

Here we use the `derive` command to generate an intermediate derivative `mag_preproc` (which for T1w images just involves a call to fslreorient2std)

In [ ]:
! banana derive data/ds000114 mri.T1wAnalysis my_banana_analysis mag_preproc --subject_ids 01 --visit_ids test --param reorient_to_std 1 --scratch work --processor single --reprocess

In [ ]:
import os.path as op
from nilearn import plotting, image
def plot_slices(path, title):
    plotting.plot_anat(image.load_img(op.join('data/ds000114', path)),
                       title=title, display_mode='z', dim=-1,
                       cut_coords=[-20, -10, 0, 10, 20, 30])

plot_slices('derivatives/my_banana_analysis/sub-01/ses-test/mag_preproc.nii.gz', 'Reorientated')

## Exercise 1

Use the Banana command line (don't forget the preceding '!' to execute the command in the underlying Bash shell instead of with Python), resample the preprocessed magnitude to 4mm isotropic and plot the results using `plot_slices`.

**Hint:** to provide a list as a parameter enclose it in \[\] and separate the items with ',', e.g. [1,2,3]

In [ ]:
## Write your solution here

In [ ]:
! banana menu mri.T1wAnalysis

In [ ]:
! banana derive data/ds000114 mri.T1wAnalysis my_banana_analysis mag_preproc --subject_ids 01 --visit_ids test --param resampled_resolution [4,4,4] --param reorient_to_std 1 --scratch work --processor single --reprocess

In [ ]:
plot_slices('derivatives/my_banana_analysis/sub-01/ses-test/mag_preproc.nii.gz', 'Resampled')

## Exercise 2

Use the `mri.T1wAnalysis` class to derive a brain extracted and brain mask from the T1-weighted image of Subject 01, using a f-threshold of 0.11 and plot the results using `plot_slices`.

**Note** Running the brain extraction can take ~5 min feel free to move on to the next notebook while you are waiting for it to finish

In [ ]:
## Write your solution here

In [ ]:
! banana derive data/ds000114 mri.T1wAnalysis my_banana_analysis brain brain_mask --parameter bet_f_threshold 0.11 --subject_ids 01 --visit_ids test --scratch work --processor single --reprocess

In [ ]:
plot_slices('derivatives/my_banana_analysis/sub-01/ses-test/brain.nii.gz', 'Brain')
plot_slices('derivatives/my_banana_analysis/sub-01/ses-test/brain_mask.nii.gz', 'Brain Mask')